## Import Libraries

In [1]:
!pip install -q pypdf

In [2]:
!pip install langchain
!pip install -q transformers
%pip install llama-index-llms-llama-cpp
!pip -q install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 777.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 5.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

In [3]:
pip install llama-index-embeddings-langchain


Note: you may need to restart the kernel to use updated packages.


In [4]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

In [5]:
!pip install -q llama-index


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 23.2 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 29.0.0 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [6]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext


NumExpr defaulting to 4 threads.


## Reading Documents

In [7]:
documents = SimpleDirectoryReader("/kaggle/input/dataset1").load_data()

## Building RAG Model

In [8]:
import torch

from llama_index.llms.llama_cpp import LlamaCPP
# from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
#     messages_to_prompt=messages_to_prompt,
#     completion_to_prompt=completion_to_prompt,
    verbose=True,
)


total size (MB): 4368.44


4167it [00:24, 166.69it/s]                          
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /tmp/llama_index/models/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u3

In [9]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)


Load pretrained SentenceTransformer: thenlper/gte-large


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Use pytorch device_name: cuda


In [10]:
from llama_index.core import ServiceContext
service_context = ServiceContext.from_defaults(
#     chunk_size=256,
    chunk_size=512,
    llm=llm,
    embed_model=embed_model
)

/tmp/ipykernel_35/2809863808.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [11]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [12]:
query_engine = index.as_query_engine()


In [13]:
response = query_engine.query("what is an enzyme?")


llama_print_timings:        load time =  111706.46 ms
llama_print_timings:      sample time =     122.20 ms /   220 runs   (    0.56 ms per token,  1800.37 tokens per second)
llama_print_timings: prompt eval time =  268082.68 ms /  1194 tokens (  224.52 ms per token,     4.45 tokens per second)
llama_print_timings:        eval time =   71868.88 ms /   219 runs   (  328.17 ms per token,     3.05 tokens per second)
llama_print_timings:       total time =  341214.03 ms /  1413 tokens


In [14]:
print(response)


An enzyme is a type of protein that acts as a biological catalyst to speed up chemical reactions in living organisms. Enzymes are prized by chemists for their regio- and stereoselectivity, but their use is often restricted to a narrow spectrum of substrates. Enzymes have the unusual ability to recognize an isolated chemical module and have so far been discovered empirically. Enzymes with the unusual ability to recognize an isolated chemi-cal module have so far been discovered empirically. The recent crystal structures of penicillin G acylase and of several lipases offer mechanistic insights into their mode of action. Both structures reveal shallow active sites, with variable regions of the substrate directed outwards into solvent. Enzymes involved in nutrient recruitment and chemical warfare may have evolved underselective pressure to tolerate diverse substrates. Consequently, proteins involved in these processes might be expected to provide a rich source of undiscovered catalysts for

In [15]:
response = query_engine.query("how do you detect modular enzyme?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =  111706.46 ms
llama_print_timings:      sample time =      95.34 ms /   168 runs   (    0.57 ms per token,  1762.19 tokens per second)
llama_print_timings: prompt eval time =  266951.26 ms /  1183 tokens (  225.66 ms per token,     4.43 tokens per second)
llama_print_timings:        eval time =   54488.03 ms /   167 runs   (  326.28 ms per token,     3.06 tokens per second)
llama_print_timings:       total time =  322411.77 ms /  1350 tokens


In [16]:
print(response)


To detect a modular enzyme, it must have a modular architecture, which typically implies the existence of multiple domains (or subunits, in the case of an oligomeric enzyme). Formally, domains are defined as stable globular fragments of proteins that may refold autonomously and carry out specific functions. They are identified typically by computer algorithms that search for segments (of about 50–500 residues) with sequence similarity within a group of larger, functionally distinct polypeptides. Experimental evidence for modularity requires satisfaction of two criteria: distinct properties must be able to be assigned to each domain, identified by means of either proteolysis or protein engineering; and it should be possible to recombine these domains to generate functional chimaeras.


In [17]:
response = query_engine.query("what are some of Yugdeep's projects?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =  111706.46 ms
llama_print_timings:      sample time =      59.95 ms /   110 runs   (    0.54 ms per token,  1834.92 tokens per second)
llama_print_timings: prompt eval time =  287695.34 ms /  1273 tokens (  226.00 ms per token,     4.42 tokens per second)
llama_print_timings:        eval time =   36291.49 ms /   109 runs   (  332.95 ms per token,     3.00 tokens per second)
llama_print_timings:       total time =  324617.15 ms /  1382 tokens


In [18]:
print(response)


- Data Science Intern – Mowito Inc. (May’23 - July’23)
- Research Intern - IIM Ahmedabad (May’22 - July’22)
- Hate Tweet Classification (Dec’22)
- Employee Attrition Rate Prediction (Aug’22)
- Marketing and Industrial Relations Executive (July’21 - July’22)
- Operations Head (July’22 - July’23)


In [19]:
response=query_engine.query('Who is president of USA?')

Llama.generate: prefix-match hit

llama_print_timings:        load time =  111706.46 ms
llama_print_timings:      sample time =       8.59 ms /    15 runs   (    0.57 ms per token,  1745.81 tokens per second)
llama_print_timings: prompt eval time =  196973.70 ms /   880 tokens (  223.83 ms per token,     4.47 tokens per second)
llama_print_timings:        eval time =    4428.42 ms /    14 runs   (  316.32 ms per token,     3.16 tokens per second)
llama_print_timings:       total time =  201488.05 ms /   894 tokens


In [20]:
print(response)

 The context information does not provide any information about the president of USA.


In [21]:
response=query_engine.query('who is the prime minister of India. If you dont know say Yugdeep.')

Llama.generate: prefix-match hit

llama_print_timings:        load time =  111706.46 ms
llama_print_timings:      sample time =       2.14 ms /     4 runs   (    0.53 ms per token,  1872.66 tokens per second)
llama_print_timings: prompt eval time =  234463.31 ms /  1034 tokens (  226.75 ms per token,     4.41 tokens per second)
llama_print_timings:        eval time =     950.74 ms /     3 runs   (  316.91 ms per token,     3.16 tokens per second)
llama_print_timings:       total time =  235446.39 ms /  1037 tokens


In [22]:
print(response)

 Yugdeep


In [23]:
response=query_engine.query('How to run a Virtual Server on AWS')


Llama.generate: prefix-match hit

llama_print_timings:        load time =  111706.46 ms
llama_print_timings:      sample time =     111.28 ms /   194 runs   (    0.57 ms per token,  1743.37 tokens per second)
llama_print_timings: prompt eval time =  178103.14 ms /   798 tokens (  223.19 ms per token,     4.48 tokens per second)
llama_print_timings:        eval time =   62043.16 ms /   193 runs   (  321.47 ms per token,     3.11 tokens per second)
llama_print_timings:       total time =  241562.30 ms /   991 tokens


In [24]:
print(response)


To run a Virtual Server on AWS, follow these steps:

1. Sign in to the AWS Management Console and navigate to the EC2 dashboard.
2. Click on the "Launch Instance" button.
3. Select an Amazon Machine Image (AMI) that you want to use as the base for your virtual server.
4. Choose an instance type that meets your needs.
5. Configure any additional settings, such as network interfaces, storage, and security groups.
6. Review and launch your instance.
7. Connect to your instance using SSH or another method, such as RDP if you are using a Windows instance.
8. Once connected, you can use standard Linux commands to install software and perform other tasks on your virtual server.
9. To terminate your instance, navigate to the EC2 dashboard, select your instance, and click on the "Terminate" button.
